In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from typing import List, Tuple

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from datasets import load_dataset

ModuleNotFoundError: No module named 'sklearn'

https://huggingface.co/datasets/Davlan/sib200 - качаем датасет отсюда

In [4]:
!git clone https://huggingface.co/datasets/Davlan/sib200

Cloning into 'sib200'...
remote: Enumerating objects: 872, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 872 (delta 12), reused 8 (delta 8), pack-reused 857 (from 1)
Receiving objects: 100% (872/872), 14.12 MiB | 9.70 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [5]:

'''
Необходимо проверить, совпадают ли категории в тренировочной, валидационной и тестовой выборках. Если нет, выводить сообщение об ошибке и RuntimeError.
В переменную categories положить все классы, которые есть в тренировочной выборке. В валидационной и тестовой оставить только категории, которые есть в тренировочной.
Конфигурация rus_Cyrl.
'''
from typing import Tuple, List
from datasets import load_dataset

def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    #конфигурация 'rus_Cyrl' шоб был русиш
    dataset = load_dataset('Davlan/sib200', 'rus_Cyrl')

    #данные из тренировочной, валидационной и тестовой выборок
    train_data = dataset['train']
    val_data = dataset['validation']
    test_data = dataset['test']

    #все уникальные категории из тренировочной выборки
    categories_train = sorted(set(train_data['category']))

    #типа категории из валидационной и тестовой выборок входят в тренировочные категории
    categories_val = sorted(set(val_data['category']))
    categories_test = sorted(set(test_data['category']))

    if not set(categories_val).issubset(categories_train):
        raise RuntimeError('Валидационная выборка содержит категории, которых нет в тренировочной выборке.') #ыыы

    if not set(categories_test).issubset(categories_train):
        raise RuntimeError('Тестовая выборка содержит категории, которых нет в тренировочной выборке.') #ыыы

    # типа те данные в валидационной и тестовой выборках, категории которых есть в тренировочной
    val_data = [(text, label) for text, label in zip(val_data['text'], val_data['category']) if label in categories_train]
    test_data = [(text, label) for text, label in zip(test_data['text'], test_data['category']) if label in categories_train]

    #преобразуем данные в формат чтобы их кушала функция возвращает функция
    X_train, y_train = train_data['text'], train_data['category']
    X_val, y_val = [x[0] for x in val_data], [x[1] for x in val_data]
    X_test, y_test = [x[0] for x in test_data], [x[1] for x in test_data]

    categories = categories_train # категории

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories


In [6]:
'''
Убрать знаки препинания, выполнить токенизацию и лемматизацию.
Вместо всех числовых токенов вставить <NUM>.
Привести к нижнему регистру.
'''
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    # NLP пайплайн (токенизация, лемматизация)
    doc = nlp_pipeline(s.lower())  # приведение к нижнему регистру !!!важно

    lemmas = []
    for token in doc:
        if token.is_punct:  # убираем знаки препинания
            continue
        if token.like_num:  # если токен похож на число, заменяем на <NUM>
            lemmas.append('<NUM>')
        else:
            lemmas.append(token.lemma_)  # лемматизация !!!!

    return ' '.join(lemmas)

In [7]:
train_data, val_data, test_data, classes_list = load_sib200_ru() # запускаем

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
# напечатать список категорий

classes_list

['entertainment',
 'geography',
 'health',
 'politics',
 'science/technology',
 'sports',
 'travel']

In [9]:
# проверить размерности всех полученных выборок

#размерности
print(f"Размер тренировочной выборки: {len(train_data[0])} текстов и {len(train_data[1])} меток")
print(f"Размер валидационной выборки: {len(val_data[0])} текстов и {len(val_data[1])} меток")
print(f"Размер тестовой выборки: {len(test_data[0])} текстов и {len(test_data[1])} меток")

#количество категорий
print(f"Количество категорий: {len(classes_list)}")



Размер тренировочной выборки: 701 текстов и 701 меток
Размер валидационной выборки: 99 текстов и 99 меток
Размер тестовой выборки: 204 текстов и 204 меток
Количество категорий: 7


In [10]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 35.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
# загрузить пайплайн ru_core_news_sm


nlp = spacy.load('ru_core_news_sm')

In [36]:
# применить normalize_text и nlp ко всем выборкам
def apply_normalization(X_data: List[str], nlp_pipeline: spacy.Language) -> List[str]:
    return [normalize_text(text, nlp_pipeline) for text in X_data]

#загружаем данные в нашу функцию
train_data, val_data, test_data, categories = load_sib200_ru()

# применяем нормализацию
X_train_normalized = apply_normalization(train_data[0], nlp)
X_val_normalized = apply_normalization(val_data[0], nlp)
X_test_normalized = apply_normalization(test_data[0], nlp)

#размерности после нормализации
print(f"Размер тренировочной выборки после нормализации: {len(X_train_normalized)} текстов")
print(f"Размер валидационной выборки после нормализации: {len(X_val_normalized)} текстов")
print(f"Размер тестовой выборки после нормализации: {len(X_test_normalized)} текстов")

Размер тренировочной выборки после нормализации: 701 текстов
Размер валидационной выборки после нормализации: 99 текстов
Размер тестовой выборки после нормализации: 204 текстов


In [13]:
# создать Pipeline для векторизации текстов и логистической регрессии



train_data, val_data, test_data, classes_list = load_sib200_ru()

#сконектим тренировочные и валидационные данные для обучения модели
X_train = train_data[0] + val_data[0]
y_train = train_data[1] + val_data[1] # делаем мы это потому что кросс-валидация типа, наверное так лучше

X_test = test_data[0]
y_test = test_data[1]

#Pipeline включающий векторизацию TF-IDF и логистическую регрессию
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # векторизация текста с помощью TF-IDF
    ('clf', LogisticRegression(max_iter=1000))  # лог регрессия
])



In [14]:
# подобрать параметры с помощью GridSearchCV

#сетка гиперпараметров для поиска (нашел в инете вроде самая распространенная тема)
param_grid = {
    'tfidf__max_df': [0.8, 0.9, 1.0],  # игнорировать слова которые встречаются в более чем 80% 90% или 100% документов
    'tfidf__min_df': [1, 5, 10],       # игнорировать слова которые встречаются менее чем в 1 5 или 10 документах
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # однословные или биграммные представления
    'clf__C': [0.01, 0.1, 1, 10],  # параметр регуляризации (обратная сила регуляризации)
    'clf__solver': ['lbfgs', 'liblinear']  # алгоритм оптимизации
}

#GridSearchCV с 5-кратной кросс-валидацией
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)


In [27]:
# обучить классификатор с перебором гиперпараметров
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(max_iter=1000))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.01, 0.1, 1, 10],
                         'clf__solver': ['lbfgs', 'liblinear'],
                         'tfidf__max_df': [0.8, 0.9, 1.0],
                         'tfidf__min_df': [1, 5, 10],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=2)

In [35]:



print("Лучшие параметры:")
print(grid_search.best_params_)

y_pred = grid_search.best_estimator_.predict(X_test)


print(classification_report(y_test, y_pred, target_names=classes_list))




Лучшие параметры:
{'clf__C': 10, 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.8, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
                    precision    recall  f1-score   support

     entertainment       0.43      0.16      0.23        19
         geography       0.86      0.35      0.50        17
            health       0.57      0.18      0.28        22
          politics       0.73      0.73      0.73        30
science/technology       0.58      0.84      0.69        51
            sports       0.68      0.52      0.59        25
            travel       0.52      0.78      0.62        40

          accuracy                           0.60       204
         macro avg       0.62      0.51      0.52       204
      weighted avg       0.61      0.60      0.57       204



In [29]:
# напечатать наилучшие параметры

print("Лучшие параметры:")
print(grid_search.best_params_)


Лучшие параметры:
{'clf__C': 10, 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.8, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}


In [30]:
# напечатать лучший F1

print("Лучший F1-Score на тестовой выборке:", classification_report(test_data[1], y_pred, target_names=categories, output_dict=True)['weighted avg']['f1-score'])


Лучший F1-Score на тестовой выборке: 0.5667368047773726


In [31]:
# напечатать размер словаря векторизованных текстов

print("Размер словаря TF-IDF:", len(grid_search.best_estimator_.named_steps['tfidf'].vocabulary_))


Размер словаря TF-IDF: 7249


In [33]:
# получить предсказания на валидационной выборке и вывести classification_report

y_val_pred = grid_search.best_estimator_.predict(X_val_normalized)
print("\nОтчет по валидационной выборке:")
print(classification_report(val_data[1], y_val_pred, target_names=categories))


Отчет по валидационной выборке:
                    precision    recall  f1-score   support

     entertainment       1.00      0.67      0.80         9
         geography       1.00      0.75      0.86         8
            health       1.00      0.73      0.84        11
          politics       1.00      0.86      0.92        14
science/technology       0.78      1.00      0.88        25
            sports       1.00      0.83      0.91        12
            travel       0.76      0.95      0.84        20

          accuracy                           0.87        99
         macro avg       0.93      0.83      0.86        99
      weighted avg       0.90      0.87      0.87        99



In [34]:
# получить предсказания на тестовой выборке и вывести classification_report

y_test_pred = grid_search.best_estimator_.predict(X_test_normalized)
print("\nОтчет по тестовой выборке:")
print(classification_report(test_data[1], y_test_pred, target_names=categories))


Отчет по тестовой выборке:
                    precision    recall  f1-score   support

     entertainment       0.75      0.16      0.26        19
         geography       1.00      0.18      0.30        17
            health       0.64      0.32      0.42        22
          politics       0.83      0.50      0.62        30
science/technology       0.49      0.80      0.61        51
            sports       0.64      0.36      0.46        25
            travel       0.41      0.72      0.53        40

          accuracy                           0.52       204
         macro avg       0.68      0.43      0.46       204
      weighted avg       0.63      0.52      0.50       204



In [21]:
# проанализировать полученные результаты и написать выводы


#ВЫВОД
Наилучшие гиперпараметры, найденные в результате поиска, показали, что модель лучше работает с униграммами слабая регуляризация C=10 позволила повысить точность классификации.
Итоговые метрики продемонстрировали, что модель хорошо справляется с более частыми категориями, такими как "Politics" и "Science/Technology", однако испытывает трудности с менее представленными в данных категориями, такими как "Entertainment" и "Health".
Среднее значение F1-score составило примерно 0.56, а точность классификации — примерно 52%.
Что является для меня не очень показателем, возможно изза датасета, воозможно я не доконца понял какие гиперпараметры нужно, либо ещё можно расширить дататсет для более редкиз категорий и можно ещё эксперементировать с ROS - тэгами